# 🏋️ CrossFit RAG - Pipeline de Ingesta

Este notebook ejecuta el pipeline completo de preparación de datos:

1. **Setup**: Instala dependencias y configura la API key
2. **Parse**: Convierte los PDFs a JSON estructurado
3. **Ingest**: Carga los JSONs en ChromaDB
4. **Verificación**: Confirma que el retrieval funciona

> ⚠️ Ejecutá las celdas **en orden**, de arriba hacia abajo.

---
## 🔑 Celda 2 - Configuración de la API key

Tenés dos opciones:

**Opción A (recomendada):** Crear un archivo `.env` en la raíz del proyecto con este contenido:
```
ANTHROPIC_API_KEY=tu_api_key_acá
```

**Opción B:** Ingresarla directamente en la celda de abajo (no recomendado si compartís el notebook).

In [1]:
import os
from dotenv import load_dotenv

# Carga la API key desde el archivo .env
load_dotenv()

# Verifica que la API key esté disponible antes de continuar
api_key = os.getenv("ANTHROPIC_API_KEY")

if api_key:
    # Muestra solo los primeros y últimos caracteres por seguridad
    print(f"✅ API key cargada correctamente: {api_key[:8]}...{api_key[-4:]}")
else:
    print("❌ No se encontró la API key.")
    print("   Creá un archivo .env con: ANTHROPIC_API_KEY=tu_key")
    print("   O descomentá la línea de abajo y pegá tu key directamente:")
    

✅ API key cargada correctamente: sk-ant-a..._wAA


---
## 📁 Celda 3 - Estructura de carpetas
Crea las carpetas necesarias si no existen y verifica que los PDFs estén en su lugar.

In [2]:
from pathlib import Path

# Crea las carpetas del proyecto si no existen
carpetas = [
    Path("data/raw"),        # Acá van los PDFs originales
    Path("data/processed"),  # Acá se guardan los JSONs procesados
    Path("chroma_db"),       # Acá persiste ChromaDB
]

for carpeta in carpetas:
    carpeta.mkdir(parents=True, exist_ok=True)
    print(f"📁 {carpeta}: OK")

print()

# Verifica que los PDFs estén en data/raw/
pdfs_encontrados = list(Path("data/raw").glob("*.pdf"))

if pdfs_encontrados:
    print(f"✅ PDFs encontrados en data/raw/:")
    for pdf in sorted(pdfs_encontrados):
        size_kb = round(pdf.stat().st_size / 1024, 1)
        print(f"   📄 {pdf.name} ({size_kb} KB)")
else:
    print("⚠️  No se encontraron PDFs en data/raw/")
    print("   Copiá tus archivos PDF ahí con los nombres:")
    print("   semana_01.pdf, semana_02.pdf, semana_03.pdf")

📁 data\raw: OK
📁 data\processed: OK
📁 chroma_db: OK

✅ PDFs encontrados en data/raw/:
   📄 ClasesCrossfit_ejemplo1.pdf (241.1 KB)
   📄 ClasesCrossfit_ejemplo2.pdf (224.2 KB)
   📄 ClasesCrossfit_ejemplo3.pdf (252.8 KB)


---
## 🔍 Celda 4 - Preview del texto extraído del PDF

Antes de parsear, verificamos que `pdfplumber` extrae el texto correctamente.
Si el texto se ve desordenado acá, hay que ajustar la extracción antes de continuar.

In [3]:
import pdfplumber

# Cambiá este nombre si tu primer PDF tiene un nombre distinto
PDF_PRUEBA = "data/raw/ClasesCrossfit_ejemplo1.pdf"

if not Path(PDF_PRUEBA).exists():
    print(f"❌ No se encontró {PDF_PRUEBA}")
    print("   Asegurate de copiar los PDFs a data/raw/ antes de continuar")
else:
    with pdfplumber.open(PDF_PRUEBA) as pdf:
        print(f"📄 Archivo: {PDF_PRUEBA}")
        print(f"   Páginas: {len(pdf.pages)}")
        print()

        # Muestra el texto de la primera página para verificar que se extrae bien
        texto_pagina_1 = pdf.pages[0].extract_text()

        if texto_pagina_1:
            print("✅ Texto extraído correctamente. Preview (primeros 500 caracteres):")
            print("-" * 50)
            print(texto_pagina_1[:500])
            print("-" * 50)
            print("\n💡 Si el texto se ve ordenado y legible, podés continuar con la celda 5.")
            print("   Si se ve desordenado, avisale al desarrollador para ajustar la extracción.")
        else:
            print("❌ No se pudo extraer texto de la página 1")
            print("   El PDF puede ser un escaneo. Contactá al desarrollador.")

📄 Archivo: data/raw/ClasesCrossfit_ejemplo1.pdf
   Páginas: 5

✅ Texto extraído correctamente. Preview (primeros 500 caracteres):
--------------------------------------------------
CLASES CROSSFIT
SEMANA 9/2
LUNES 9/2
DÍA 1:
CORE: 3 rounds
• TGU 2/2 (Pesado)
• Side Plank Pallof press 12/12
FUERZA
• Back Squat 5x5 (AHAP)
• Principiante: Box squat / goblet
WOD: EMOM 18’
12/10 cal remo
12 KB front rack lunges
10 Devil Press double db
ACCESORIOS – 3 rounds
• Nordic o curl femoral 10
• Monster walk 20 mts
• Vuelos Laterales
MARTES 10/2
DÍA 2
CORE: 3 rounds
• Farmer carry pesado 20 m
--------------------------------------------------

💡 Si el texto se ve ordenado y legible, podés continuar con la celda 5.
   Si se ve desordenado, avisale al desarrollador para ajustar la extracción.


In [4]:
# Importa las funciones del script de parseo
# Asegurate de que parse_routine.py esté en la carpeta scripts/
import sys
sys.path.insert(0, "scripts")  # Agrega la carpeta scripts al path de Python

from scripts.parse_routine import parsear_pdf

# Define los pares (PDF de entrada, JSON de salida)
semanas_a_procesar = [
    ("data/raw/ClasesCrossfit_ejemplo1.pdf", "data/processed/ClasesCrossfit_ejemplo1.json"),
    ("data/raw/ClasesCrossfit_ejemplo2.pdf", "data/processed/ClasesCrossfit_ejemplo2.json"),
    ("data/raw/ClasesCrossfit_ejemplo3.pdf", "data/processed/ClasesCrossfit_ejemplo3.json"),
]

resultados = []

for ruta_pdf, ruta_json in semanas_a_procesar:
    if not Path(ruta_pdf).exists():
        print(f"⚠️  Saltando {ruta_pdf} (no encontrado)")
        continue

    try:
        semana = parsear_pdf(ruta_pdf, ruta_json)
        resultados.append(semana)
        print(f"🏋️  {semana.semana_id}: {len(semana.dias)} días procesados\n")
    except Exception as e:
        print(f"❌ Error en {ruta_pdf}: {e}\n")

print(f"\n✅ Parseo completado: {len(resultados)} semanas procesadas")


📂 Procesando: data/raw/ClasesCrossfit_ejemplo1.pdf
📄 PDF procesado: 5 páginas, 1918 caracteres extraídos
🤖 Enviando al LLM para estructurar...
DEBUG respuesta cruda:
```json
{
  "semana_id": "semana_2024_W06",
  "fecha_inicio": "2024-02-05",
  "dias": [
    {
      "dia": "Lunes",
      "fecha": "2024-02-05",
      "tipo_bloque_principal": "FUERZA",
      "core": {
        "rondas": 3,
        "ejercicios": [
          {
            "nombre": "TGU",
            

✔️  Validando estructura con Pydantic...
✅ Rutina guardada en: data/processed/ClasesCrossfit_ejemplo1.json
🏋️  semana_2024_W06: 6 días procesados


📂 Procesando: data/raw/ClasesCrossfit_ejemplo2.pdf
📄 PDF procesado: 4 páginas, 2242 caracteres extraídos
🤖 Enviando al LLM para estructurar...
❌ Error en data/raw/ClasesCrossfit_ejemplo2.pdf: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01khrxh07aebtrcx6ksske2ybe` service tier `on_demand` on tokens per day (TP

---
## 🔎 Celda 6 - Inspeccionar el JSON generado

Verificamos que el JSON tenga la estructura correcta antes de cargarlo en ChromaDB.

In [5]:
import json

JSON_PRUEBA = "data/processed/ClasesCrossfit_ejemplo1.json"

with open(JSON_PRUEBA, "r", encoding="utf-8") as f:
    semana = json.load(f)

print(f"✅ JSON cargado correctamente")
print(f"   Semana ID : {semana['semana_id']}")
print(f"   Inicio    : {semana['fecha_inicio']}")
print(f"   Días      : {len(semana['dias'])}")
print()

for dia in semana["dias"]:
    meta = dia["metadata"]
    wod = dia["wod"]
    bp = dia.get("bloque_principal") or {}
    
    # Usa "N/A" si el campo es None para evitar el error de formato
    tipo = dia.get("tipo_bloque_principal") or "N/A"
    movimiento = bp.get("movimiento") or "N/A"
    
    print(f"  📅 {dia['dia']:10} | {tipo:7} | "
          f"WOD: {wod['formato']:8} | "
          f"Fuerza: {movimiento:20} | "
          f"Intensidad: {meta['intensidad_estimada']}")

✅ JSON cargado correctamente
   Semana ID : semana_2024_W06
   Inicio    : 2024-02-05
   Días      : 6

  📅 Lunes      | FUERZA  | WOD: EMOM     | Fuerza: Back Squat           | Intensidad: alta
  📅 Martes     | OLY     | WOD: Rounds   | Fuerza: Push Jerk            | Intensidad: alta
  📅 Miercoles  | FUERZA  | WOD: AMRAP    | Fuerza: Deadlift             | Intensidad: alta
  📅 Jueves     | FUERZA  | WOD: EMOM     | Fuerza: Strict Press         | Intensidad: alta
  📅 Viernes    | OLY     | WOD: AMRAP    | Fuerza: Snatch               | Intensidad: alta
  📅 Sabado     | N/A     | WOD: For Time | Fuerza: N/A                  | Intensidad: alta


---
## 🗄️ Celda 7 - Cargar en ChromaDB

Vectoriza los JSONs y los carga en la base de datos vectorial.

In [9]:
from scripts.ingest import inicializar_chroma, cargar_todas_las_semanas

# Inicializa ChromaDB y obtiene la colección
coleccion = inicializar_chroma()

# Carga todos los JSONs de data/processed/ en ChromaDB
total = cargar_todas_las_semanas(coleccion)

print(f"\n{'='*50}")
print(f"📊 Semanas cargadas en esta ejecución : {total}")
print(f"📚 Total de documentos en ChromaDB    : {coleccion.count()}")
print(f"{'='*50}")

✅ ChromaDB inicializado en: ./chroma_db
📚 Colección 'rutinas_crossfit': 2 documentos existentes

📂 Encontrados 3 archivos JSON para cargar
✅ Cargada: semana_2024_W06 (6 días, 2114 chars)
✅ Cargada: semana_2025_W07 (5 días, 2037 chars)
✅ Cargada: semana_2024_W06 (5 días, 1969 chars)

📊 Semanas cargadas en esta ejecución : 3
📚 Total de documentos en ChromaDB    : 2


---
## ✅ Celda 8 - Verificar el retrieval

Hacemos búsquedas de prueba para confirmar que ChromaDB devuelve resultados coherentes.
Esta es la validación más importante antes de pasar al pipeline RAG.

In [10]:
# Pruebas de búsqueda semántica en ChromaDB
# Cambiá las queries para probar distintos escenarios

queries_prueba = [
    "rutina con sentadilla y trabajo de piernas",
    "día con snatch y movimientos olímpicos",
    "WOD for time con burpees y saltos",
    "trabajo de empuje con press de banca",
]

print("🔍 Resultados de búsquedas de prueba:\n")

for query in queries_prueba:
    resultados = coleccion.query(
        query_texts=[query],
        n_results=min(2, coleccion.count()),
        include=["metadatas", "distances"]
    )

    print(f"  🔎 Query: '{query}'")

    for meta, dist in zip(resultados["metadatas"][0], resultados["distances"][0]):
        similitud = round(1 - dist, 3)
        print(f"     → {meta['semana_id']} | similitud: {similitud} | "
              f"olímpicos: {meta['movimientos_olimpicos'] or 'ninguno'}")
    print()

🔍 Resultados de búsquedas de prueba:

  🔎 Query: 'rutina con sentadilla y trabajo de piernas'
     → semana_2024_W06 | similitud: 0.296 | olímpicos: clean, snatch
     → semana_2025_W07 | similitud: 0.285 | olímpicos: clean, snatch

  🔎 Query: 'día con snatch y movimientos olímpicos'
     → semana_2025_W07 | similitud: 0.308 | olímpicos: clean, snatch
     → semana_2024_W06 | similitud: 0.293 | olímpicos: clean, snatch

  🔎 Query: 'WOD for time con burpees y saltos'
     → semana_2025_W07 | similitud: 0.256 | olímpicos: clean, snatch
     → semana_2024_W06 | similitud: 0.221 | olímpicos: clean, snatch

  🔎 Query: 'trabajo de empuje con press de banca'
     → semana_2024_W06 | similitud: 0.309 | olímpicos: clean, snatch
     → semana_2025_W07 | similitud: 0.267 | olímpicos: clean, snatch



---
## 📋 Celda 9 - Resumen final

Estado completo de la base de datos vectorial.

In [11]:
# Muestra todos los documentos cargados en ChromaDB con su metadata
todos = coleccion.get(include=["metadatas"])

print(f"📚 Base de datos ChromaDB - '{coleccion.name}'")
print(f"   Total de semanas: {len(todos['ids'])}")
print()
print(f"{'ID':<20} {'Inicio':<12} {'Días':<6} {'Olímpicos':<20} {'Patrones'}")
print("-" * 75)

for doc_id, meta in zip(todos["ids"], todos["metadatas"]):
    print(
        f"{doc_id:<20} "
        f"{meta['fecha_inicio']:<12} "
        f"{meta['total_dias']:<6} "
        f"{(meta['movimientos_olimpicos'] or 'ninguno'):<20} "
        f"{meta['patrones_fuerza']}"
    )

print()
print("🎉 Pipeline de ingesta completado exitosamente.")
print("   El siguiente paso es construir retriever.py y generator.py")

📚 Base de datos ChromaDB - 'rutinas_crossfit'
   Total de semanas: 2

ID                   Inicio       Días   Olímpicos            Patrones
---------------------------------------------------------------------------
semana_2024_W06      2024-02-05   5      clean, snatch        OLY, empuje, sentadilla
semana_2025_W07      2025-02-16   5      clean, snatch        OLY, empuje, sentadilla

🎉 Pipeline de ingesta completado exitosamente.
   El siguiente paso es construir retriever.py y generator.py
